## MVP Engenharia de Dados - Notebook 4: Análise (SQL)

**Objetivo:** Responder às perguntas de negócio consultando o modelo de dados final (`default.gold_vendas_flat_model`).

**Período Analisado:** 01/01/2025 a 13/11/2025.

**As 8 Perguntas de Negócio:**

**1. Quais são os 10 produtos mais vendidos (em quantidade)?**
> **Nota de Análise:** O item "VALOR DO SERVIÇO" foi excluído das consultas de produtos por se tratar de uma taxa operacional.

**2. Dentre os diversos tipos de opções, quais os Grupos que vendem mais?**
> **Nota de Análise:** Agrupei os produtos de acordo com a categorização dos alimentos e bebidas.

**3.  Quais são os 10 produtos menos vendidos (em quantidade)?**

**4.  Qual o ranking de faturamento por PDV (bar)?**

**5.  Qual é o faturamento por hora do dia (para identificar picos)?**

**6.  Qual é o valor médio por nota fiscal (Ticket Médio)?**

**7.  Quais são os 10 sócios (ID_SOCIO) que mais consumiram (em valor)?**

**8.  Qual a divisão do faturamento total entre os tipos de consumo ("A Faturar" vs. "Pago na Hora")?**

In [0]:
%sql
-- Verificação do período dos dados
SELECT 
    MIN(DATA_HORA) as Data_Inicial,
    MAX(DATA_HORA) as Data_Final,
    DATEDIFF(MAX(DATA_HORA), MIN(DATA_HORA)) as Qtd_Dias
FROM default.gold_vendas_flat_model;

#### 1. Quais são os 10 produtos mais vendidos (em quantidade)?

In [0]:
%sql
-- 1. Top 10 Produtos
SELECT 
  NOME_PRODUTO,
  NOME_GRUPO,
  SUM(QUANTIDADE) as Total_Vendido
FROM default.gold_vendas_flat_model
WHERE NOME_GRUPO <> 'TAXA DE SERVIÇO'
GROUP BY NOME_PRODUTO, NOME_GRUPO
ORDER BY Total_Vendido DESC
LIMIT 10;

Databricks visualization. Run in Databricks to view.

** Insight**

1.  **Itens de Alto Volume:** A lista dos 10 produtos mais vendidos é dominada por itens de necessidade e conveniência (como água, refrigerantes), sendo que CAFE EXPRESSO e AGUA MINALBA SEM GAS 510ML lideram o volume de unidades vendidas.

#### 2. Dentre os diversos tipos de opções, quais os Grupos que vendem mais?

In [0]:
%sql
-- 2. Ranking de Grupos
SELECT 
  NOME_GRUPO,
  COUNT(DISTINCT COD_PRODUTO) as Qtd_Produtos_No_Grupo,
  SUM(FATURAMENTO_LIQUIDO) as Faturamento_por_Grupo
FROM default.gold_vendas_flat_model
WHERE NOME_GRUPO <> 'TAXA DE SERVIÇO'
GROUP BY NOME_GRUPO
ORDER BY Faturamento_por_Grupo DESC;

In [0]:
%sql
-- 2. Mix de Vendas Detalhado
SELECT 
  CASE 
    -- 1. BEBIDAS
    WHEN NOME_GRUPO LIKE '1.%' THEN 'BEBIDAS (S/ ÁLCOOL)'
    WHEN NOME_GRUPO LIKE '2.%' THEN 'BEBIDAS (ALCOÓLICAS)'
    
    -- 2. COMIDAS
    WHEN NOME_GRUPO LIKE '3.01%' THEN 'AVES'
    WHEN NOME_GRUPO LIKE '3.02%' THEN 'CARNES'
    WHEN NOME_GRUPO LIKE '3.03%' THEN 'PEIXES E FRUTOS DO MAR'
    WHEN NOME_GRUPO LIKE '3.04%' THEN 'MASSAS E RISOTOS'
    WHEN NOME_GRUPO LIKE '3.05%' THEN 'OVOS E OMELETES'
    WHEN NOME_GRUPO LIKE '3.06%' THEN 'ENTRADAS E SALADAS'
    WHEN NOME_GRUPO LIKE '3.07%' THEN 'PETISCOS E PORÇÕES'
    WHEN NOME_GRUPO LIKE '3.08%' THEN 'GUARNIÇÕES'
    WHEN NOME_GRUPO LIKE '3.09%' THEN 'JAPONÊS'
    WHEN NOME_GRUPO LIKE '3.10%' THEN 'SOPAS'
    WHEN NOME_GRUPO LIKE '3.11%' THEN 'MOLHOS'
    WHEN NOME_GRUPO LIKE '3.12%' THEN 'INFANTIL'
    WHEN NOME_GRUPO LIKE '3.13%' THEN 'LANCHES E SANDUÍCHES'
    WHEN NOME_GRUPO LIKE '3.14%' THEN 'CAFÉ DA MANHÃ'
    WHEN NOME_GRUPO LIKE '3.15%' THEN 'BUFFET E EVENTOS'
    
    -- 3. SOBREMESAS
    WHEN NOME_GRUPO LIKE '3.16%' OR NOME_GRUPO LIKE '3.17%' THEN 'SOBREMESAS'
    WHEN NOME_GRUPO LIKE '3.18%' OR NOME_GRUPO LIKE '3.19%' THEN 'SORVETES'
    
    -- 4. OUTROS GRUPOS
    WHEN NOME_GRUPO LIKE '3.20%' THEN 'ENCOMENDAS'
    WHEN NOME_GRUPO LIKE '3.21%' THEN 'SUGESTÕES DO CHEF'
    WHEN NOME_GRUPO LIKE '4.%' THEN 'BOMBONIERE'
    WHEN NOME_GRUPO LIKE '5.%' THEN 'TABACARIA'
    
    -- 5. TAXAS
    WHEN NOME_GRUPO LIKE 'Taxa%' OR NOME_GRUPO LIKE 'TAXA%' THEN 'TAXA DE SERVIÇO'
    
    ELSE 'OUTROS'
  END AS Grupos,
  
  SUM(FATURAMENTO_LIQUIDO) as Faturamento_por_Grupo,
  ROUND(SUM(FATURAMENTO_LIQUIDO) * 100 / SUM(SUM(FATURAMENTO_LIQUIDO)) OVER (), 1) as Percentual_Faturamento
  
FROM default.gold_vendas_flat_model
WHERE NOME_GRUPO <> 'TAXA DE SERVIÇO'
GROUP BY Grupos
ORDER BY Faturamento_por_Grupo DESC;

Databricks visualization. Run in Databricks to view.

**Insight**

**Principais Constatações:**

1.  **Concentração de Receita:** O grupo `BUFFET E EVENTOS` responde por 18.7% de todo o faturamento.
2.  **Peso das Bebidas:** O consumo de bebidas (Alcoólicas + Não Alcoólicas) representa o segundo maior faturamento, superando categorias tradicionais como Carnes e Peixes.
3.  **Dispersão da Cauda Longa:** Existe uma grande pulverização de receitas em categorias de apoio (Molhos, Guarnições, Tabacaria), que individualmente contribuem pouco para o montante final.

#### 3. Quais são os 10 produtos menos vendidos (em quantidade)?

In [0]:
%sql
-- 3. Top 10 Produtos Menos Vendidos
SELECT 
  NOME_PRODUTO,
  NOME_GRUPO,
  SUM(QUANTIDADE) as Total_Vendido
FROM default.gold_vendas_flat_model
WHERE NOME_GRUPO <> 'TAXA DE SERVIÇO' 
GROUP BY NOME_PRODUTO, NOME_GRUPO
ORDER BY Total_Vendido ASC
LIMIT 10;

Databricks visualization. Run in Databricks to view.

**Insight**

1.  **Itens de baixo volume:** Todos os 10 produtos listados registraram apenas uma única unidade vendida (Quantidade = 1) em quase 11 meses de operação.

%md
#### 4. Qual o ranking de faturamento por PDV (bar)?

In [0]:
%sql
-- 4. Ranking de Faturamento por Ponto de Venda (PDV)
SELECT 
  NOME_PDV,
  SUM(FATURAMENTO_LIQUIDO) as Faturamento_por_PDV
FROM default.gold_vendas_flat_model
WHERE NOME_GRUPO <> 'TAXA DE SERVIÇO'
GROUP BY NOME_PDV
ORDER BY Faturamento_por_PDV DESC;

Databricks visualization. Run in Databricks to view.

**Insight**

1.  A receita do estabelecimento é concentrada no principal ponto de venda, sendo o **Bar Misto** o responsável pela maior parte do faturamento total.

2.  O PDV **Caramanchao I** apresentou a menor contribuição total para a receita do período.

#### 5. Qual é o faturamento por hora do dia (para identificar picos)?

In [0]:
%sql
-- 5. Qual é o faturamento por hora do dia (para identificar picos)?
SELECT 
  hour(DATA_HORA) as Hora_do_Dia,
  SUM(FATURAMENTO_LIQUIDO) as Faturamento_Total
FROM default.gold_vendas_flat_model
WHERE NOME_GRUPO <> 'TAXA DE SERVIÇO'
GROUP BY Hora_do_Dia
ORDER BY Hora_do_Dia ASC;

Databricks visualization. Run in Databricks to view.

**Insight**

1.  O gráfico revela um comportamento de consumo concentrado no meio da tarde, com o pico máximo de faturamento às 16:00h, seguido imediatamente pelas 15:00h.

3.  Observa-se um baixo volume de vendas nas primeiras horas da manhã (06h às 10h) e, notavelmente, uma queda acentuada no período noturno (20h às 23h).

#### 6. Qual é o valor médio por nota fiscal (Ticket Médio)?

In [0]:
%sql
-- 6. Ticket Médio (Valor Médio por Venda)
SELECT 
  ROUND(SUM(FATURAMENTO_LIQUIDO) / COUNT(DISTINCT NUM_NFCE), 2) as Ticket_Medio_Reais
FROM default.gold_vendas_flat_model
WHERE NOME_GRUPO <> 'TAXA DE SERVIÇO';

**Insight**

1.  O ticket médio de R$ 121,49 indica o perfil de consumo típico de um cliente a cada visita.

#### 7. Quais são os 10 sócios (ID_SOCIO) que mais consumiram (em valor)?

In [0]:
%sql
-- 7. Ranking de Clientes (Top 10 Sócios por Valor Gasto)
SELECT 
  ID_SOCIO,
  SUM(FATURAMENTO_LIQUIDO) as Total_Gasto
FROM default.gold_vendas_flat_model
WHERE ID_SOCIO IS NOT NULL 
  AND ID_SOCIO <> '' 
  AND ID_SOCIO <> '0' -- Filtra clientes não identificados
GROUP BY ID_SOCIO
ORDER BY Total_Gasto DESC
LIMIT 10;

Databricks visualization. Run in Databricks to view.

**Insight:**

1.  O ranking destaca os clientes de maior valor agregado para o estabelecimento, sendo o sócio de ID 910 o maior consumidor individual do período.
2.  Os 10 sócios listados possuem um volume de gastos significativamente superior à média (Ticket Médio), indicando alta frequência ou alto valor por pedido.
3.  A consulta considerou apenas sócios identificados, excluindo vendas anônimas.

#### 8. Qual a divisão do faturamento total entre os tipos de consumo ("A Faturar" vs. "Pago na Hora")?

In [0]:
%sql
-- 8. Divisão por Tipo de Pagamento
SELECT 
  TIPO_PAGAMENTO,
  SUM(FATURAMENTO_LIQUIDO) as Faturamento_Total,
  ROUND(SUM(FATURAMENTO_LIQUIDO) * 100 / SUM(SUM(FATURAMENTO_LIQUIDO)) OVER (), 1) as Percentual_Faturamento
FROM default.gold_vendas_flat_model
WHERE NOME_GRUPO <> 'TAXA DE SERVIÇO'
GROUP BY TIPO_PAGAMENTO
ORDER BY Faturamento_Total DESC;

Databricks visualization. Run in Databricks to view.

**Insight:**

1.  A operação do estabelecimento é fortemente baseada em vendas com cobrança posterior. A categoria SOCIO_A_FATURAR responde por uma esmagadora maioria, totalizando 90.4% do faturamento líquido.
2.  A receita de AVULSO_PAGO_NA_HORA (que representa o pagamento à vista) é marginal, respondendo por apenas 9.6% do total.
3.  O fluxo de caixa e a saúde financeira dependem quase inteiramente da cobrança e liquidação dos valores "A Faturar", sugerindo um controle de crédito robusto e bem estabelecido.